In [1]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility


import random
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D,BatchNormalization
from keras.regularizers import l2, activity_l2
from keras.optimizers import RMSprop
from keras import backend as K
from datasets.tidr import load_data


Using TensorFlow backend.


In [2]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(10)]) - 1
    for d in range(10):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, 10)
            dn = (d + inc) % 10
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

def create_compare(x_train, x_ref):
    pairs = []
    labels = []
    for i in xrange(25):
        for j in xrange(120):
            pairs += [[x_train[120*i+j], x_ref[i]]]
            
    return np.array(pairs)



def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()

    seq.add(Convolution2D(64, 5, 5, border_mode='same',
                input_shape=input_shape))
    seq.add(Activation('relu'))
    seq.add(Convolution2D(64, 5, 5))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(5, 5), strides=(2, 2)))
    seq.add(Dropout(0.25))

    seq.add(Convolution2D(128, 3, 3, border_mode='same'))
    seq.add(BatchNormalization())
    seq.add(Activation('relu'))
    seq.add(Convolution2D(128, 3, 3))
    seq.add(BatchNormalization())
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
    seq.add(Dropout(0.25))

    seq.add(Flatten())
    seq.add(Dense(512))
    seq.add(Activation('relu'))
    return seq


def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()

In [3]:
DistortImg, DistortLabel, RefImg, RefLabel, ScoreLabel = load_data()




# create training+test positive and negative pairs
'''digit_indices = [np.where(y_train == i)[0] for i in range(10)]
tr_pairs, tr_y = create_pairs(X_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(10)]
te_pairs, te_y = create_pairs(X_test, digit_indices)'''

all_pairs = create_compare(DistortImg, RefImg)

X_train = all_pairs[1:2001]
X_test =  all_pairs[2001:3000]

Y_train = ScoreLabel[1:2001]
Y_test = ScoreLabel[2001:3000]

In [16]:
ScoreLabel = np.asarray(ScoreLabel)
ScoreLabel = ScoreLabel / 10
ScoreLabel 

array([ 0.0551429,  0.0556757,  0.0494444, ...,  0.0462857,  0.0402778,
        0.0277143])

In [11]:
X_train /= 255
X_test /= 255
input_dim = 96,128
nb_epoch = 20
input_shape = (96,128,1)

# network definition
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model(input=[input_a, input_b], output=distance)


In [90]:
#x_train1 = np.array([X_train[:,0],])

x_train1 = np.expand_dims(X_train[:,0], axis=3)
x_train2 = np.expand_dims(X_train[:,1], axis=3)
x_test1 = np.expand_dims(X_test[:,0], axis=3)
x_test2 = np.expand_dims(X_test[:,1], axis=3)

x_train1.shape

(2000, 96, 128, 1)

In [91]:
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms)
model.fit( [x_train1, x_train2], Y_train,
          validation_data=([x_test1, x_test2], Y_test),
          batch_size=32,
          nb_epoch=nb_epoch)

Train on 2000 samples, validate on 999 samples
Epoch 1/20
 288/2000 [===>..........................] - ETA: 726s - loss: nan

KeyboardInterrupt: 